# Covid 19 Case Study [*Ongoing*]

### Dataset:
The datasets are collected from Kaggle and are just for demonstration purposes. I do not guarantee accuracy or completeness of this dataset. The construction of an accurate dataset is under way and will be completed soon. 
<br>The dataset links are:

1) Time series Covid-19 World Dataset-- https://www.kaggle.com/sudalairajkumar/novel-corona-virus-2019-dataset
<br>
2) World Population Dataset [1960-2018]-- https://www.kaggle.com/imdevskp/world-population-19602018/data

[Note-- The population dataset(2) is from 2018 while the Covid-19 dataset(1) has its last updated value on 1st June, 2020. Thus there are bounds to be major discrepancies in the data-comparison. Also there are minor adjustments made by me on the datasets to help with the plot.]

### Usage: 

The interactive plot below is a visual representation of the dataset used. The following interactions are possible on the plot:

1) A single click on any of the countries mentioned in the plot leads to detailed plots about the pandemic situation in these countries. A hover function displays the total number of Covid-19 cases along with the country name and the date on the cases recorded. As of now, the date is a fixed value, fixed on the last date of the dataset update.
<br><br>
2) The single click leads to an animated plot where the animation can be started by pressing the "|>" button. Following the animated plot, there is the world plot once again and further line and pie charts can be found which are self explanatory. The pie chart with population statistics contains labels of "Non-Covid", "Covid", and "deaths from Covid". It can be observed interactively by toggling the desired labels in the legend of the chart ( with right clicks).
<br><br>
3) More than one countries can be selected by using the "box select" tool at the top right corner of the world plot and then dragging the mouse pointer over the contents to be selected. This  would open up a new plot sequence where the world plot is at top-most, followed by a "Country tally". This country wise plot can be observed interactively with the help of the "compare data on hover tool" wherein it would show the data from all the curves simultaneously when hovered over the plot with the mouse pointer. <br>[Here the over-65 percentage tells the percentage of the population which is over 65 in these countries via a pie chart. Again, the data ( as far as my observation) is not complete and does not show all countries. As a result, an empty plot or a plot with a single country can be observed at times. This also happens with the population reports in singly selected countries]
<br><br>
4) More than one country can be selected also by using the "lasso select" tool at the top right corner of the world plot and then dragging the mouse pointer over the contents to be selected.
<br><br>
5) Another functionality to select more than a single country for analysis is similar to how we select files. Keeping ctrl pressed on the keyboard, the countries can be selected one by one. The important thing here is that after selecting the desired countries, a new click has to be made to any other country--to ensure the selection is complete for further actions to take effect ( as plotly doesn't yet support recording only keyboard events)
<br><br>
6) The world map provided with all the plot sequences can be used to navigate between different countries or groups of countries.




In [2]:
from urllib.request import urlopen
import json
from IPython.core.display import HTML
import cufflinks as cf
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, Markdown, clear_output
import plotly.io as pio
from plotly.subplots import make_subplots
from ipywidgets import HBox,VBox
import numpy as np
import plotly.graph_objs as go
import plotly.express as px

#     for i in points.point_inds:
#         c[i] = '#bae2be'
#         s[i] = 20
        
#         with f.batch_update():
#             scatter.marker.color = c
#             scatter.marker.size = s

# counties=[]    
# with open('Countries.json','r') as response:
#     counties = json.load(response)
    
#df = px.data.gapminder()
#print(df)
raw_data = pd.read_csv("time_series_covid_19_confirmed.csv")
death_data = pd.read_csv("time_series_covid_19_deaths.csv")
population_data=pd.read_csv("pop_worldometer_data.csv")
over_data=pd.read_csv("population_above_age_65_percentage_long.csv")
over_data=over_data.loc[over_data['Year'] == 2017]

#exit(1)
# for i,data in enumerate(raw_data['iso3']):
# 	if data=='USA':
# 		raw_data.drop(i,axis=0,inplace=True)

#print(raw_data['Province_State'])

#print(df)

#print(raw_data)
#print()


var=raw_data.keys().values[-1]
#var=var[0]

f=go.FigureWidget(px.scatter_geo(raw_data,scope="world",locationmode = 'country names',locations="Country/Region",
                    color=var,color_continuous_scale='blues_r',
                 hover_name="Country/Region"))


p=[]
flag=0
def controller(points):
    global p,fk,f1
    country=[]
    for i in points:
        count=raw_data.loc[[i]]['Country/Region']
        country.append(count.values[0])
    #print(country)    
    country_data=call_me_for_more(country,1)
    country_deaths=call_me_for_deaths(country,1)
    #print(country_data)
    fig1 = px.line(country_data, x="dates", y="cases", color='country')
    fig2 = px.line(country_deaths, x="dates", y="deaths", color='country')
    #fig.update_xaxes(rangeslider_visible=True)
    fig=go.FigureWidget(px.line(country_data,title="Country tally--Cases", x="dates", y="cases", color='country'))
    fig1=go.FigureWidget(px.line(country_deaths,title="Country tally--Deaths", x="dates", y="deaths", color='country'))
    fid=HBox([fig,fig1])
    count=0
    for i,country in enumerate(population_data[population_data.keys()[0]]):
        
        if country in country_data["country"].values:
                #print("sdf")
                #print(data)
                #print(population_data.iloc[i][0:2].transpose())
                df_new = population_data.iloc[[i]]
                df_new2 = over_data.iloc[[i]]
                if count==0:
                    df_store=(df_new)
                    df_store2=(df_new2)
                    #print("once")
                else:
                    
                    df_store = pd.concat([df_store, df_new])
                    df_store2 = pd.concat([df_store2, df_new2])
                    #print(df_store)
                    #print(df_new)
                    #print("many")
                count+=1
    #print(df_store)
    fig_pop1 = go.FigureWidget(px.pie(df_store, values=df_store.keys()[4], names=df_store.keys()[0], title='Population Density'))
    fig_pop2 = go.FigureWidget(px.pie(df_store2, values=df_store2.keys()[2], names=df_store2.keys()[0], title='Over-65 Percentage'))
    fig_pop3 = go.FigureWidget(px.pie(df_store, values=df_store.keys()[1], names=df_store.keys()[0], title='Total Population'))
    
    pop=HBox([fig_pop1,fig_pop3])
    pop=VBox([pop,fig_pop2])
    
    f1=VBox([fk,fid,pop])
    clear_output(wait=True)
    
    display(f1)

        
flag=0
def update_point(trace, points, selector):
    global flag,fk
    #print("Updating plot")
    #c = list(scatter.marker.name)
     
    for i in points.point_inds:
        #print(c[i])
        #print(i)
        if (selector.ctrl==True):
            #print("called")
            p.append(i)
            flag+=1
            return
        if flag>=1 and selector.ctrl==False:
            controller(p)
            flag=0
            return  
        
                
        
        country=raw_data.loc[[i]]['Country/Region']
        country=country.values[0]
        country_data=call_me_for_more(country)
        country_death=call_me_for_deaths(country)
        
        
        
        #fig.show()
        
        
        
        fig_drawing = go.Figure(
                data=[go.Scatter(x=[0], y=[0],mode="markers")],
                layout=go.Layout(
                yaxis=dict(range=[0, country_data['cases'].values[-1]], autorange=False),
                xaxis=dict(range=[0, 132], autorange=False),
                xaxis_title="Day",
                yaxis_title="Cases",
                title=country_data['country'].values[-1]+"- Daywise Analysis (Cases)",
                updatemenus=[dict(
                    type="buttons",
                    buttons=[dict(label="|>",
                                  method="animate",
                                  args=[None,{"frame": {"duration": 20, "redraw": False},}])])]
            ),
             frames=[go.Frame(
                data=[go.Scatter(
                    x=np.arange(i),
                    y=country_data["cases"][0:i],
                    mode="lines",
                    marker=dict(color="blue", size=10))])

                for i in range(len(country_data["cases"].values))]
        )
        
        
        fig_drawing.update_layout(transition_duration=0)
        
        fig_cases = px.line(country_data, x="dates", y="cases", color='country')
        
        fig_deaths=px.line(country_death, x="dates", y="deaths", color='country')
        
        
        #fig.show()
        
        fig_drawing_death=go.Figure(
                data=[go.Scatter(x=[0], y=[0],mode="markers")],
                layout=go.Layout(
                yaxis=dict(range=[0, country_death['deaths'].values[-1]], autorange=False),
                xaxis=dict(range=[0, 132], autorange=False),
                xaxis_title="Day",
                yaxis_title="Deaths",
                title=country_death['country'].values[-1]+"- Daywise Analysis (Deaths)",
                updatemenus=[dict(
                    type="buttons",
                    buttons=[dict(label="|>",
                                  method="animate",
                                  args=[None, {"frame": {"duration": 20, "redraw": False},}])])]
            ),
             frames=[go.Frame(
                data=[go.Scatter(
                    x=np.arange(i),
                    y=country_death["deaths"][0:i],
                    mode="lines",
                    marker=dict(color="red", size=10))])

                for i in range(len(country_death["deaths"].values))]
        )
        
        fig_drawing_death.update_layout(transition_duration=0)
        
        
        fig_cases.update_layout(title="Date Wise Analysis (Cases)")
        fig1=go.FigureWidget(fig_cases)
        fig_deaths.update_layout(title="Date Wise Analysis (Deaths)")
        fig2=go.FigureWidget(fig_deaths)
        fig_box=HBox([fig1,fig2])
        f=VBox([fk,fig_box])
        clear_output(wait=True)
        
        #subplots_drawing=go.Figure(make_subplots(rows=1, cols=2,subplot_titles=("Cases v/s Days","Deaths v/s Days")))
        #subplots_drawing.add_trace(fig_drawing,row=1,col=1)
        #subplots_drawing.add_trace(fig_drawing_death,row=1,col=2)
        #display(HBox([fig_drawing,fig_drawing_death]))
        #subplots_drawing.show()
        fig_drawing.update_layout(
             autosize=False,
            width=800,
            height=500
#             margin=dict(
#                 l=20,
#                 r=20,
#                 b=20,
#                 t=20,
#                 pad=5
#             )
            )
        fig_drawing_death.update_layout(
            autosize=False,
            width=800,
            height=500
#             margin=dict(
#                 l=20,
#                 r=20,
#                 b=20,
#                 t=20,
#                 pad=5
#             )
            )
        fig_drawing.update_yaxes(automargin=True)
        fig_drawing_death.update_yaxes(automargin=True)
        fig_drawing.show()
        fig_drawing_death.show()
        fig_pop=go.Figure()
        count_pop=0
        display(f)
        
        for i,country in enumerate(population_data[population_data.keys()[0]]):
            if country in country_data["country"].values:
                count_pop=population_data.iloc[[i]][population_data.keys()[1]].values
                #print(count_pop)
                
                break
                
        if count_pop!=0:
            deaths=country_death['deaths'].values[-1]
            
            cases=country_data['cases'].values[-1]
            labels=['Non-Covid','Covid','Deaths from Covid']
            nc=count_pop[0]-cases-deaths
            values=[nc,cases,deaths]
            #print(cases,deaths,count_pop)
            fig_pop = go.Figure(data=[go.Pie(labels=labels, values=values)])
            fig_pop.update_layout(title="Population statistics for "+country_data['country'].values[1])
            fig_pop.show()
        else:
            fig_pop.update_layout(title="Population report is Unavailable at the moment!")
            fig_pop.show()
            
        
        
        
        
        
        
        #print(trace)
        #print(selector)
        
def call_me_for_more(country,label=0):
    df_store=pd.DataFrame()
    count=0
    if label==0:
        for i,data in enumerate(raw_data['Country/Region']):
            if data==country:
                df_new = raw_data.iloc[i][4:]
                df_new=df_new.T
                df_new = df_new.reset_index()
                #df_new = df_new.set_index(['date', 'cases'])
                df_new=df_new.rename(columns={df_new.keys()[0]: 'dates',df_new.keys()[1]:'cases'})
                
                df_new['country']=data
#                 x=[]
#                 y=[]
#                 for case in df_new['cases']:
#                     x.append(np.random.randint(1,100,case))
#                     y.append(np.random.randint(1,100,case))
                
#                 df_new['x_rand']=x
#                 df_new['y_rand']=y
                
                #print(df_new.keys())
                return df_new
    else:
        for i,data in enumerate(raw_data['Country/Region']):
            if data in country:
                #print(data)
                df_new = raw_data.iloc[i][4:]
                df_new=df_new.T
                df_new = df_new.reset_index()
                #df_new = df_new.set_index(['date', 'cases'])
                df_new=df_new.rename(columns={df_new.keys()[0]: 'dates',df_new.keys()[1]:'cases'})
                df_new['country']=data
                if count==0:
                    df_store=(df_new)
                    #print("once")
                else:
                    df_store = pd.concat([df_store, df_new])
                    #print(df_new)
                    #print("many")
                count+=1
                #print(df_new.keys())
        #print(df_store)
        return df_store

def call_me_for_deaths(country,label=0):
    df_store=pd.DataFrame()
    count=0
    if label==0:
        for i,data in enumerate(death_data['Country/Region']):
            if data==country:
                df_new = death_data.iloc[i][4:]
                df_new=df_new.T
                df_new = df_new.reset_index()
                #df_new = df_new.set_index(['date', 'cases'])
                df_new=df_new.rename(columns={df_new.keys()[0]: 'dates',df_new.keys()[1]:'deaths'})
                
                df_new['country']=data
#                 x=[]
#                 y=[]
#                 for case in df_new['cases']:
#                     x.append(np.random.randint(1,100,case))
#                     y.append(np.random.randint(1,100,case))
                
#                 df_new['x_rand']=x
#                 df_new['y_rand']=y
                
                #print(df_new.keys())
                return df_new
    else:
        for i,data in enumerate(death_data['Country/Region']):
            if data in country:
                #print(data)
                df_new = death_data.iloc[i][4:]
                df_new=df_new.T
                df_new = df_new.reset_index()
                #df_new = df_new.set_index(['date', 'cases'])
                df_new=df_new.rename(columns={df_new.keys()[0]: 'dates',df_new.keys()[1]:'deaths'})
                df_new['country']=data
                if count==0:
                    df_store=(df_new)
                    #print("once")
                else:
                    df_store = pd.concat([df_store, df_new])
                    #print(df_new)
                    #print("many")
                count+=1
                #print(df_new.keys())
        #print(df_store)
        return df_store
            
        
        
        
def selection_fn(trace,points,selector):
    country=[]
    global fk,f1
    for i in points.point_inds:
        #print(c[i])
        #print(i)
        
        count=raw_data.loc[[i]]['Country/Region']
        country.append(count.values[0])
    #print(country)    
    country_data=call_me_for_more(country,1)
    country_deaths=call_me_for_deaths(country,1)
    
    #print(country_data)
    #fig = go.FigureWidget(px.line(country_data, x="dates", y="cases", color='country'))
    clear_output(wait=True)
    fig=go.FigureWidget(px.line(country_data,title="Country tally--Cases", x="dates", y="cases", color='country'))
    fig1=go.FigureWidget(px.line(country_deaths,title="Country tally--Deaths", x="dates", y="deaths", color='country'))
    df_store=0
    df_store2=0
    count=0
    
    for i,country in enumerate(population_data[population_data.keys()[0]]):
        
        if country in country_data["country"].values:
                #print("sdf")
                #print(data)
                #print(population_data.iloc[i][0:2].transpose())
                df_new = population_data.iloc[[i]]
                df_new2 = over_data.iloc[[i]]
                if count==0:
                    df_store=(df_new)
                    df_store2=(df_new2)
                    #print("once")
                else:
                    
                    df_store = pd.concat([df_store, df_new])
                    df_store2 = pd.concat([df_store2, df_new2])
                    #print(df_store)
                    #print(df_new)
                    #print("many")
                count+=1
    count=0
    for i,country in enumerate(over_data[over_data.keys()[0]]):
        
        if country in country_data["country"].values:
                #print("sdf")
                #print(data)
                #print(population_data.iloc[i][0:2].transpose())
                
                df_new2 = over_data.iloc[[i]]
                if count==0:
                    
                    df_store2=(df_new2)
                    #print("once")
                else:
                    df_store2 = pd.concat([df_store2, df_new2])
                    #print(df_store)
                    #print(df_new)
                    #print("many")
                count+=1
    #print(df_store)
    fig_pop1 = go.FigureWidget(px.pie(df_store, values=df_store.keys()[4], names=df_store.keys()[0], title='Population Density'))
    fig_pop2 = go.FigureWidget(px.pie(df_store2, values=df_store2.keys()[2], names=df_store2.keys()[0], title='Over-65 Percentage'))
    fig_pop3 = go.FigureWidget(px.pie(df_store, values=df_store.keys()[1], names=df_store.keys()[0], title='Total Population'))
    
    pop=HBox([fig_pop1,fig_pop3])
    pop=VBox([pop,fig_pop2])
            
    fig=HBox([fig,fig1])
    f1=VBox([fk,fig,pop])
    #print(country_data)
    display(f1)
    
    
    
    #fig.update_xaxes(rangeslider_visible=True)
    #clear_output(wait=True)
#     scatter=f.data[0]
#     scatter.on_click(update_point)
#     scatter.on_selection(selection_fn)
    
    #box_me=HBox([f,fig])
    #return "edfgbnwesdffg"
    
scatter=f.data[0]
scatter.on_click(update_point)

scatter.on_selection(selection_fn)

f.update_traces(textposition='top center')

f.update_layout(
    height=500,
    title_text='COVID 19 WORLD PLOT'
)

fk=f
f=VBox([f])

display(f)

# with open('p_graph.html', 'a') as f1:
#     f1.write(fig.to_html(full_html=False, include_plotlyjs='cdn'))
#     f1.write(fig.to_html(full_html=False, include_plotlyjs='cdn'))
#     f1.write(fig.to_html(full_html=False, include_plotlyjs='cdn'))
widgets.Output()

    'data': [{'geo': 'geo',
              'hovertemplate': ('<b>%{hovertext}</b>…

Output()